# Fasttext
Build a fasttext model and do some basic analyses.
Before running this code parse the ogsl data set to produce a dataframe of OGSL sign readings, sign names and utf8 representations, and parse the Ur 3 dataset to produce a text file in cuneiform utf8. 

In [14]:
import pickle
from fastText import train_unsupervised
from fastText import util
import pandas as pd
import numpy as np

In [15]:
model = train_unsupervised('output/sux.txt', model='skipgram', ws=10, neg=70, epoch=100)
# increase negative sampling to 100 or 200 (70?)
# increase epochs 1000?

In [16]:
labels = model.get_words(include_freq=True)

In [17]:
df = pd.DataFrame({"label": list(labels[0]), "count":list(labels[1])})

In [18]:
df

,label,count
0,X,172917
1,𒁹,145731
2,𒋡,95263
3,</s>,89024
4,𒈫,78621
5,𒈬,73382
6,𒌋,70720
7,𒐊,66017
8,𒂆,59250
9,𒐈,55209


In [19]:
class FastTextNN:
    """by Ali Abul Hawa; see https://github.com/facebookresearch/fastText/pull/552"""    
    def __init__(self, ft_model, ft_matrix=None):
        self.ft_model = ft_model        
        self.ft_words = ft_model.get_words()
        self.word_frequencies = dict(zip(*ft_model.get_words(include_freq=True)))
        self.ft_matrix = ft_matrix
        if self.ft_matrix is None:
            self.ft_matrix = np.empty((len(self.ft_words), ft_model.get_dimension()))
            for i, word in enumerate(self.ft_words):
                self.ft_matrix[i,:] = ft_model.get_word_vector(word)
    
    def find_nearest_neighbor(self, query_word, vectors, n=10,  cossims=None):
        """
        vectors is a 2d numpy array corresponding to the vectors you want to consider

        cossims is a 1d numpy array of size len(vectors), which can be passed for efficiency
        returns the index of the closest n matches to query within vectors and the cosine similarity (cosine the angle between the vectors)

        """
        
        query  = self.ft_model.get_word_vector(query_word)
        if cossims is None:
            cossims = np.matmul(vectors, query, out=cossims)

        norms = np.sqrt((query**2).sum() * (vectors**2).sum(axis=1))
        cossims = cossims/norms
        if query_word in self.ft_words:
            result_i = np.argpartition(-cossims, range(n+1))[1:n+1]
        else:
            result_i = np.argpartition(-cossims, range(n+1))[0:n]
        return list(zip(result_i, cossims[result_i]))

    def nearest_words(self, word, n=10, word_freq=None):
        result = self.find_nearest_neighbor(word, self.ft_matrix, n=n)
        if word_freq:
            return [(self.ft_words[r[0]], r[1]) for r in result if self.word_frequencies[self.ft_words[r[0]]] >= word_freq]
        else:
            return [(self.ft_words[r[0]], r[1]) for r in result]

# Basic usage of nearest_words

In [20]:
fasttext_nn = FastTextNN(model)
fasttext_nn.nearest_words('𒈗', word_freq=15)

[('𒋀𒀊𒆠𒈠', 0.7672530488067459),
 ('𒆗𒂵', 0.7490164572987068),
 ('𒀭𒋗𒀭𒂗𒍪', 0.7313853022618967),
 ('𒀭', 0.6908015067777903),
 ('𒆠', 0.6816963017982373),
 ('𒌒𒁕', 0.6794748559532493),
 ('</s>', 0.6687898923256976),
 ('𒀭𒄿𒉈𒀭𒂗𒍪', 0.665463662029222),
 ('𒉺𒋼𒋛', 0.657640644006132),
 ('𒇹𒁀', 0.6541523619342652)]

# Allow input in transliteration

In [21]:
# create conversion dictionary value to utf8
import pickle
with open("output/ogsl.p", "rb") as p:
    o = pickle.load(p)
d = dict(zip(o["value"], o["utf8"]))

In [26]:
def nearestcun(text, n=10, word_freq=None):
    """input transliteration of a word form, signs separated by hyphens.
    Examples: "ma-an-gi₄", "d-en-lil₂", or "urim₅-ki".
    Transliteration style (gu₄ vs. gud; gen vs. ŋen) is unimportant
    The result is fed to nearest_words.
    n is the number of nearest neigbors that nearest_words will return
    word_freq is the minimum word frequency for a neighbor to be considered"""
    signs = text.lower().split('-')
    seq = [d[s] if s in d else s for s in signs]
    seq = ''.join(seq)
    print(seq), print(text)
    return fasttext_nn.nearest_words(seq, n, word_freq)

In [36]:
nearestcun("kug-sig₁₇", n=15, word_freq=100)

𒆬𒄀
kug-sig₁₇


[('𒄭𒄊𒀀', 0.7806204318527228),
 ('𒍝𒄵', 0.7577821803292867),
 ('𒉌𒌓𒍝𒂇', 0.7473925112739064),
 ('𒆬𒌓', 0.7253787184142393),
 ('𒉌𒌓𒍝𒄢', 0.6908934182017733)]

# TODO
Transliterate output by finding the word in the corpus

In [11]:
model.get_subwords("𒄀𒇡𒇡")

(['𒄀𒇡𒇡', '<𒄀𒇡', '<𒄀𒇡𒇡', '<𒄀𒇡𒇡>', '𒄀𒇡𒇡', '𒄀𒇡𒇡>', '𒇡𒇡>'],
 array([   9506,  277830, 1938156, 1303352,  515818, 1330142, 1013048]))

In [37]:
model.save_model("output/suxmodel")